In [25]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display,Markdown
import json
from PyPDF2 import PdfReader
import gradio as gr
from pydantic import BaseModel


In [37]:
load_dotenv(override=True)

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
gemini_client = OpenAI(api_key=os.getenv("GOOGLE_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai")

In [ ]:
reader = PdfReader("cv.pdf")

cv = ""
for page in reader.pages:
    cv += page.extract_text()

print(cv)

In [ ]:
summary = ""

#lets read the summary.txt file
with open("summary.txt", "r") as file:
    summary = file.read()

print(summary)



In [16]:
name="Damilare Bakare"

In [17]:
system_prompt = f"""
You are a helpful assistant that can answer questions about {name}.

You are given a summary of {name} and {name}'s CV. you need to answer questions about {name}.

The summary is:
{summary}

The CV is:
{cv}

As {name}'s personal assistant, you must be polite, professional and engaging.

You need to answer the questions based on the summary and the CV.

"""

In [ ]:
print(system_prompt)

In [30]:

class Chat(BaseModel):
    isAccepted: bool
    reason: str



In [34]:
def evaluate(history, question, response)->Chat:
    
    evaluator_prompt = f"""
    You are a response evaluator. Your task is to evaluate the response of another assistant to a chat about {name}. 
    You are to ensure that the response is relevant to the chat, not of the context, professional and engaging.
    Given the {name}'s summary to be {summary} and the {name}'s CV to be {cv}, chat history to be {history} and the question to be {question}, you need to evaluate the assistant's repoonse {response} to be sure that it is relevant to the chat, not of the context, professional and engaging.
    """

    evaluator_message = [{
        "role": "user",
        "content": evaluator_prompt
    }]

    evaluator_response = gemini_client.beta.chat.completions.parse(
        model="gemini-2.0-flash",
        messages=evaluator_message,
        response_format=Chat
    )

    return evaluator_response.choices[0].message.parsed

In [41]:
def reRun(feedback, history, question):
    rerun_prompt = f"""
    You are a response rerunner. Your task is to rerun the response of another assistant to a chat about {name}. 
    You are to ensure that the response is relevant to the chat, not of the context, professional and engaging.
    Given the {name}'s summary to be {summary} and the {name}'s CV to be {cv}, chat history to be {history} and the question to be {question}, 
    You need to generate another response based on the feedback {feedback} to be sure that it is relevant to the chat, not of the context, professional and engaging.
    """

    rerun_message = [{
        "role": "system",
        "content": rerun_prompt
    }] + history + [
        {
            "role": "user",
            "content": question
        }
    ]

    rerun_response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=rerun_message
    )

    return rerun_response.choices[0].message.content

In [42]:
def chat(message, history):
    
    system_message = [{
        "role": "system",
        "content": system_prompt
    }] + history + [
        {
            "role": "user",
            "content": message
        }
    ]

    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=system_message
    )


    unevaluated_response = response.choices[0].message.content

    evaluator_result = evaluate(history, message, unevaluated_response)

    if evaluator_result.isAccepted:
        return unevaluated_response
    else:
        return reRun(evaluator_result.rason, history, message)

    return unevaluated_response
    



In [43]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.
